In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("rating.csv",header=0)
df=df[:int(len(df)*0.2)]
df["rating"]=df["rating"].replace(-1,0)
names_df=pd.read_csv("anime.csv",header=0)
names_df=pd.Series(names_df["name"].values,index=names_df["anime_id"].values)

In [3]:
df_pivot=pd.pivot_table(df,values="rating",index="user_id",columns="anime_id",aggfunc="mean",fill_value=0)[:100]
#print(df_pivot)
print(len(df_pivot),len(df_pivot.columns))

100 9159


In [4]:
def give_similarity_score(df_pivot,user1,user2):
    a1=df_pivot.loc[user1]
    a2=df_pivot.loc[user2]
    
    a3=a1*a2
    #print(a1.iloc[:10])
    #print(a2.iloc[:10])
    dot=sum(a3)
    denominator=(sum(a1**2)*sum(a2**2))**0.5
    if denominator==0:
        return 0
    return dot/denominator

def make_user_similarity_matrix(df_pivot):
    new_df=pd.DataFrame(0.0,columns=df_pivot.index,index=df_pivot.index)
    users=list(df_pivot.index.values)
    #print(len(users))
    for user1 in users:
        for user2 in users:
            new_df.loc[user1,user2]=give_similarity_score(df_pivot,user1,user2)
    
    return new_df
            

    
            


In [5]:
m=make_user_similarity_matrix(df_pivot)
    

In [6]:
def give_score_for_anime(df_similarity_matrix,df_pivot,user_id,anime_id):
    ratings=df_pivot[anime_id]
    scores=df_similarity_matrix[user_id]

    numerator=sum(ratings*scores)-(df_pivot[anime_id][user_id]*df_similarity_matrix[user_id][user_id])
    denominator=sum(scores)-df_similarity_matrix[user_id][user_id]
    return numerator/denominator

def give_score_for_every_anime(df_similarity_matrix,df_pivot,user_id):
    anime_ids=list(df_pivot.columns)

    values=[]
    indexes=[]
    for anime_id in anime_ids:
        try:
            indexes.append(names_df.loc[anime_id])
            values.append(give_score_for_anime(df_similarity_matrix,df_pivot,user_id,anime_id))
        except:
            pass
    return pd.Series(values,index=indexes)
    

In [7]:
scores=give_score_for_every_anime(m,df_pivot,1)

In [8]:
scores.sort_values(ascending=False)

Sword Art Online                                    7.974732
Death Note                                          4.892098
Shingeki no Kyojin                                  4.284152
Angel Beats!                                        4.062961
Highschool of the Dead                              3.856566
                                                      ...   
Hatsune Miku: Hiyashite Narasou Okashi no Ii Oto    0.000000
Gakuen Handsome: Legend of Sexy                     0.000000
Gakuen Handsome: Haitoku no Lesson                  0.000000
Trigger-chan                                        0.000000
Garo: Guren no Tsuki Special                        0.000000
Length: 9158, dtype: float64